In [1]:

OPENAI_API_KEY=""


In [2]:
import pandas as pd
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

# Load dataset
file_path = "data/contact_center_conversations_no_sentiment.csv"
df = pd.read_csv(file_path)


In [25]:
df.head(5)

,Intent,Conversation,Mistake
0,Account Issues,"[{'speaker': 'Customer', 'text': 'Hi, I was ch...",System
1,Payment,"[{'speaker': 'Customer', 'text': 'Hi, I was ch...",Agent
2,Payment,"[{'speaker': 'Customer', 'text': 'I need to ca...",System
3,Subscription,"[{'speaker': 'Customer', 'text': ""I reset my p...",System
4,Subscription,"[{'speaker': 'Customer', 'text': 'I need to ca...",Customer


In [56]:
# df['Mistake']=df['Mistake'].fillna('System')
#df.to_csv("data/contact_center_conversations_no_sentiment.csv", index=False)

In [62]:
# Define the enhanced LLM-based prompt
prompt_template = PromptTemplate(
    input_variables=["conversation"],
    template="""
    Analyze the following customer service conversation and provide structured insights:
    
    Conversation:
    "{conversation}"
    
    
    Extract the following information and return the response in **valid JSON format** with keys: intent, sentiment_trend, resolution_status, fault_analysis, key_themes, contextual_insights, aggregated_insights, and summarization.
    - Intent of the customer (Billing, Payment, Support, etc.)
    - Sentiment progression throughout the conversation ("start": "<sentiment_at_beginning>", "mid": "<sentiment_at_midpoint>", "end": "<sentiment_at_end>")
    - Whether the issue was resolved (Yes/No/In_Progress)
    - Who was responsible for any negative sentiment during conversation ("primary_responsible": "<customer_or_agent>", "reason": "<brief_explanation>")
    - Key themes and concerns mentioned in the conversation
    - Conversion patterns and effectiveness of issue resolution
    - Insights related to intent trends and sentiment shifts
    - Analysis of fault responsibility and common causes
    - Aggregated insights and summarization of conversation patterns and outcomes
    - Knowledge generation from key themes, intent trends, and sentiment shifts for actionable insights
    
    The response should look exactly like this:
    {{
        "intent": "<intent>",
        "sentiment_trend": {{
            "start": "<sentiment_at_beginning>",
            "mid": "<sentiment_at_midpoint>",
            "end": "<sentiment_at_end>"
        }},
        "resolution_status": "<Yes/No/In_Progress>",
        "fault_analysis": {{
            "primary_responsible": "<customer_or_agent>",
            "reason": "<brief_explanation_of_why_the_issue_occurred>"
        }},
        
        "key_themes": ["<theme1>", "<theme2>"],
        
        "conversion_analysis": "<analysis_of_conversion_patterns>",
        
        "intent_trends": "<insights_on_intent_trends>",
        
        "sentiment_shift_analysis": "<analysis_of_sentiment_shifts>",
        
        "resolution_effectiveness": "<evaluation_of_issue_resolution>",
        
        "fault_trends": "<common_causes_and_responsibilities>",
        
        "aggregated_insights": "<aggregated_insights_from_conversation_patterns>",
        
        "summarization": "<summarized_overview_of_customer_interactions>",
        
        "knowledge_insights": "<actionable_knowledge_generated_from_conversation_data>"
    }}
    
    NOTE: You should never generate any supporting sentence beyond output format at any cost.  
    """
)


In [61]:

# Initialize LLM (GPT-4 or other models)
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=OPENAI_API_KEY,temperature=0.5)
chain = LLMChain(llm=llm, prompt=prompt_template)


In [63]:
# Initialize an empty list to store processed data
processed_data = []
for conversation in df['Conversation'].tolist():
    response = chain.run(conversation=conversation)
    print("*********", response, '********')
    try:
        structured_response = json.loads(response)

        # Extract values from the response
        intent = structured_response.get("intent", None)
        sentiment_start = structured_response.get("sentiment_trend", {}).get("start", None)
        sentiment_mid = structured_response.get("sentiment_trend", {}).get("mid", None)
        sentiment_end = structured_response.get("sentiment_trend", {}).get("end", None)
        resolution_status = structured_response.get("resolution_status", None)
        fault_primary = structured_response.get("fault_analysis", {}).get("primary_responsible", None)
        fault_reason = structured_response.get("fault_analysis", {}).get("reason", None)
        key_themes = structured_response.get("key_themes", [])
        conversion_analysis = structured_response.get("conversion_analysis", None)
        intent_trends = structured_response.get("intent_trends", None)
        sentiment_shift_analysis = structured_response.get("sentiment_shift_analysis", None)
        resolution_effectiveness = structured_response.get("resolution_effectiveness", None)
        fault_trends = structured_response.get("fault_trends", None)
        aggregated_insights = structured_response.get("aggregated_insights", None)
        summarization = structured_response.get("summarization", None)
        knowledge_insights = structured_response.get("knowledge_insights", None)

        # Append the extracted data as a dictionary
        processed_data.append({
            "Aggregated_Insights": aggregated_insights,
            "Summarization": summarization,
            "Knowledge_Insights": knowledge_insights,
            "Conversation": conversation,
            "Intent": intent,
            "Sentiment_Start": sentiment_start,
            "Sentiment_Mid": sentiment_mid,
            "Sentiment_End": sentiment_end,
            "Resolution_Status": resolution_status,
            "Fault_Primary_Responsible": fault_primary,
            "Fault_Reason": fault_reason,
            "Key_Themes": key_themes,
            "Conversion_Analysis": conversion_analysis,
            "Intent_Trends": intent_trends,
            "Sentiment_Shift_Analysis": sentiment_shift_analysis,
            "Resolution_Effectiveness": resolution_effectiveness,
            "Fault_Trends": fault_trends
        })
    except json.JSONDecodeError:
        print("Invalid JSON response:", response)
    except json.JSONDecodeError:
        print("Invalid JSON response:", response)

********* {
    "intent": "Billing",
    "sentiment_trend": {
        "start": "Neutral",
        "mid": "Neutral",
        "end": "Positive"
    },
    "resolution_status": "Yes",
    "fault_analysis": {
        "primary_responsible": "System",
        "reason": "The system mistakenly charged the customer twice for the same bill"
    },
    "key_themes": ["Billing Issue", "Double Charge", "Refund"],
    "conversion_analysis": "The conversation was straightforward and effective. The customer's issue was resolved promptly in a single interaction, which is ideal.",
    "intent_trends": "The customer's intent was clear from the beginning - to resolve a billing issue. The agent quickly understood and addressed the issue.",
    "sentiment_shift_analysis": "The customer's sentiment remained neutral during the conversation and turned positive after the issue was resolved.",
    "resolution_effectiveness": "The issue was effectively resolved with the agent promptly processing a refund for the 

In [64]:

# Save results to a CSV file
output_df = pd.DataFrame(processed_data)
output_df.to_csv('data/Intent_sentiment_fault_insights_summaries.csv', index=False)
